In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
import numpy as np
import random
from plotly import tools
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot, plot 
import plotly.figure_factory as ff
import plotly.graph_objs as go 

In [54]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

# 首先查看 train 文件中的内容
df_train

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0
...,...,...,...,...,...,...
35990,35991,NaN,Zimbabwe,2020-05-11,36.0,4.0
35991,35992,NaN,Zimbabwe,2020-05-12,36.0,4.0
35992,35993,NaN,Zimbabwe,2020-05-13,37.0,4.0
35993,35994,NaN,Zimbabwe,2020-05-14,37.0,4.0


In [3]:
df_test

,ForecastId,Province_State,Country_Region,Date
0,1,NaN,Afghanistan,2020-04-02
1,2,NaN,Afghanistan,2020-04-03
2,3,NaN,Afghanistan,2020-04-04
3,4,NaN,Afghanistan,2020-04-05
4,5,NaN,Afghanistan,2020-04-06
...,...,...,...,...
13454,13455,NaN,Zimbabwe,2020-05-10
13455,13456,NaN,Zimbabwe,2020-05-11
13456,13457,NaN,Zimbabwe,2020-05-12
13457,13458,NaN,Zimbabwe,2020-05-13


In [4]:
df_train.isna().sum()

Id                    0
Province_State    20700
Country_Region        0
Date                  0
ConfirmedCases        0
Fatalities            0
dtype: int64

In [55]:
df_train = df_train.fillna("")
df_train

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,,Afghanistan,2020-01-22,0.0,0.0
1,2,,Afghanistan,2020-01-23,0.0,0.0
2,3,,Afghanistan,2020-01-24,0.0,0.0
3,4,,Afghanistan,2020-01-25,0.0,0.0
4,5,,Afghanistan,2020-01-26,0.0,0.0
...,...,...,...,...,...,...
35990,35991,,Zimbabwe,2020-05-11,36.0,4.0
35991,35992,,Zimbabwe,2020-05-12,36.0,4.0
35992,35993,,Zimbabwe,2020-05-13,37.0,4.0
35993,35994,,Zimbabwe,2020-05-14,37.0,4.0


In [8]:
df_train.describe()

,Id,ConfirmedCases,Fatalities
count,35995.000000,35995.000000,35995.000000
mean,17998.000000,3683.508737,243.560217
std,10391.005806,18986.978708,1832.966999
min,1.000000,0.000000,0.000000
25%,8999.500000,0.000000,0.000000
50%,17998.000000,19.000000,0.000000
75%,26996.500000,543.000000,7.000000
max,35995.000000,345813.000000,33998.000000


In [35]:
df_countries = df_train.groupby(["Country_Region","Province_State","Date"])["ConfirmedCases"].sum()
df_countries = df_countries.groupby(["Country_Region", "Province_State"]).max()
df_countries = df_countries.groupby(["Country_Region"]).sum().sort_values(ascending=False)
df_countries = df_countries.head(20)

In [37]:
fig = px.bar(df_countries, x=df_countries.index, y='ConfirmedCases', labels={'x':'Country'},
             color="ConfirmedCases", color_continuous_scale=px.colors.sequential.Bluered)
fig.update_layout(title_text='国家历史最高确诊数')
fig.show()

In [46]:
# 首先过滤出所有美国的记录，并取省份、日期和确诊数，死亡数等字段
df_usa_records = df_train.loc[df_train["Country_Region"]=="US", ["Province_State","Date", "ConfirmedCases", "Fatalities"]]

# 将上述记录按日期维度聚合，同时这会抛弃省份维度
df_usa_records = df_usa_records.groupby("Date").sum()

# 重置索引（自动添加序号索引），否则会用 date 作为索引，不方便画图
df_usa_records = df_usa_records.reset_index()

# 查看
df_usa_records

,Date,ConfirmedCases,Fatalities
0,2020-01-22,0.0,0.0
1,2020-01-23,0.0,0.0
2,2020-01-24,0.0,0.0
3,2020-01-25,0.0,0.0
4,2020-01-26,0.0,0.0
...,...,...,...
110,2020-05-11,1347710.0,80677.0
111,2020-05-12,1369403.0,82371.0
112,2020-05-13,1390235.0,84114.0
113,2020-05-14,1417603.0,85893.0


In [44]:
fig = px.bar(df_usa_records,x='Date', y='ConfirmedCases', color="ConfirmedCases", color_continuous_scale=px.colors.sequential.Magma)
fig.update_layout(title_text='美国随时间确诊病例数')
fig.show()

In [47]:
fig = px.bar(df_usa_records,x='Date', y='Fatalities', color="Fatalities", color_continuous_scale=px.colors.sequential.Magma)
fig.update_layout(title_text='美国随时间死亡病例数')
fig.show()

In [48]:
df_brz_records = df_train.loc[df_train["Country_Region"]=="Brazil", ["Province_State","Date", "ConfirmedCases", "Fatalities"]]
df_brz_records = df_brz_records.groupby("Date").sum()
df_brz_records = df_brz_records.reset_index()

fig = px.bar(df_brz_records,x='Date', y='ConfirmedCases', color="ConfirmedCases", color_continuous_scale=px.colors.sequential.Magma)
fig.update_layout(title_text='巴西随时间确诊病例数')
fig.show()

In [49]:
fig = px.bar(df_brz_records,x='Date', y='Fatalities', color="Fatalities", color_continuous_scale=px.colors.sequential.Magma)
fig.update_layout(title_text='巴西随时间死亡病例数')
fig.show()

In [57]:
# 传入日期，将其用 - 分割，并返回第一部分，即年
def get_year(date_str):
    comps = date_str.split("-")
    return int(comps[0])

# 传入日期，将其用 - 分割，并返回第二部分，即月
def get_month(date_str):
    comps = date_str.split("-")
    return int(comps[1])

# 传入日期，将其用 - 分割，并返回第三部分，即日
def get_day(date_str):
    comps = date_str.split("-")
    return int(comps[2])

# 分别对 Date 字段 apply 上述三个函数，并用结果新建对应的 列
    df_train["Year"] = df_train.Date.apply(get_year)
    df_train["Month"] = df_train.Date.apply(get_month)
    df_train["Day"] = df_train.Date.apply(get_day)

# 查看
df_train



,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,Year,Month,Day
0,1,,Afghanistan,2020-01-22,0.0,0.0,2020,1,22
1,2,,Afghanistan,2020-01-23,0.0,0.0,2020,1,23
2,3,,Afghanistan,2020-01-24,0.0,0.0,2020,1,24
3,4,,Afghanistan,2020-01-25,0.0,0.0,2020,1,25
4,5,,Afghanistan,2020-01-26,0.0,0.0,2020,1,26
...,...,...,...,...,...,...,...,...,...
35990,35991,,Zimbabwe,2020-05-11,36.0,4.0,2020,5,11
35991,35992,,Zimbabwe,2020-05-12,36.0,4.0,2020,5,12
35992,35993,,Zimbabwe,2020-05-13,37.0,4.0,2020,5,13
35993,35994,,Zimbabwe,2020-05-14,37.0,4.0,2020,5,14


In [58]:
df_train["Country_Region"] = df_train["Country_Region"] + df_train["Province_State"]
df_train["Country_Region"].value_counts()

ChinaLiaoning                      115
Egypt                              115
Burundi                            115
USDelaware                         115
Panama                             115
                                  ... 
CanadaNew Brunswick                115
ChinaJiangsu                       115
Congo (Kinshasa)                   115
FranceSaint Pierre and Miquelon    115
Colombia                           115
Name: Country_Region, Length: 313, dtype: int64

In [59]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df_train["Country_Region"] = encoder.fit_transform(df_train["Country_Region"])
df_train

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,Year,Month,Day
0,1,,0,2020-01-22,0.0,0.0,2020,1,22
1,2,,0,2020-01-23,0.0,0.0,2020,1,23
2,3,,0,2020-01-24,0.0,0.0,2020,1,24
3,4,,0,2020-01-25,0.0,0.0,2020,1,25
4,5,,0,2020-01-26,0.0,0.0,2020,1,26
...,...,...,...,...,...,...,...,...,...
35990,35991,,312,2020-05-11,36.0,4.0,2020,5,11
35991,35992,,312,2020-05-12,36.0,4.0,2020,5,12
35992,35993,,312,2020-05-13,37.0,4.0,2020,5,13
35993,35994,,312,2020-05-14,37.0,4.0,2020,5,14


In [60]:
df_train_final = df_train[["Country_Region", "Year", "Month", "Day"]]
labels = df_train.ConfirmedCases

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(df_train_final, labels)


LinearRegression()

In [75]:
# 导入 xgboost
from xgboost import XGBRegressor

# 创建 xgboost，并配置参数
xgb = XGBRegressor(n_estimators = 2500 , random_state = 0 , max_depth = 27)

# 对我们刚才准备的特征进行训练
xgb.fit(df_train_final, labels)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=27,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=2500, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [74]:
model.predict([[0,2020,1,22]])

array([-5450.89060662])

In [78]:
xgb.predict([[0,2020,1,22]])

TypeError: Input data can not be a list.

In [62]:
model.coef_

array([  13.89363918,    0.        , 3677.4090561 ,  113.55346441])

In [64]:
df_test = df_test.fillna("")


df_test["Year"] = df_test.Date.apply(get_year)
df_test["Month"] = df_test.Date.apply(get_month)
df_test["Day"] = df_test.Date.apply(get_day)

df_test["Country_Region"] = df_test["Country_Region"] + df_test["Province_State"]
df_test


,ForecastId,Province_State,Country_Region,Date,Year,Month,Day
0,1,,Afghanistan,2020-04-02,2020,4,2
1,2,,Afghanistan,2020-04-03,2020,4,3
2,3,,Afghanistan,2020-04-04,2020,4,4
3,4,,Afghanistan,2020-04-05,2020,4,5
4,5,,Afghanistan,2020-04-06,2020,4,6
...,...,...,...,...,...,...,...
13454,13455,,Zimbabwe,2020-05-10,2020,5,10
13455,13456,,Zimbabwe,2020-05-11,2020,5,11
13456,13457,,Zimbabwe,2020-05-12,2020,5,12
13457,13458,,Zimbabwe,2020-05-13,2020,5,13


In [66]:
df_test["Country_Region"] = encoder.fit_transform(df_test["Country_Region"])


In [67]:
df_test_final = df_test[["Country_Region", "Year", "Month", "Day"]]
model.predict(df_test_final)

array([ 3310.26727349,  3423.82073789,  3537.3742023 , ...,
       12458.02639762, 12571.57986203, 12685.13332644])

In [68]:
df_test["predict_confirm"] = model.predict(df_test_final)
df_test

,ForecastId,Province_State,Country_Region,Date,Year,Month,Day,predict_confirm
0,1,,0,2020-04-02,2020,4,2,3310.267273
1,2,,0,2020-04-03,2020,4,3,3423.820738
2,3,,0,2020-04-04,2020,4,4,3537.374202
3,4,,0,2020-04-05,2020,4,5,3650.927667
4,5,,0,2020-04-06,2020,4,6,3764.481131
...,...,...,...,...,...,...,...,...
13454,13455,,312,2020-05-10,2020,5,10,12230.919469
13455,13456,,312,2020-05-11,2020,5,11,12344.472933
13456,13457,,312,2020-05-12,2020,5,12,12458.026398
13457,13458,,312,2020-05-13,2020,5,13,12571.579862


In [69]:
df_train[df_train.Date >= "2020-04-02"]

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,Year,Month,Day
71,72,,0,2020-04-02,273.0,6.0,2020,4,2
72,73,,0,2020-04-03,281.0,6.0,2020,4,3
73,74,,0,2020-04-04,299.0,7.0,2020,4,4
74,75,,0,2020-04-05,349.0,7.0,2020,4,5
75,76,,0,2020-04-06,367.0,11.0,2020,4,6
...,...,...,...,...,...,...,...,...,...
35990,35991,,312,2020-05-11,36.0,4.0,2020,5,11
35991,35992,,312,2020-05-12,36.0,4.0,2020,5,12
35992,35993,,312,2020-05-13,37.0,4.0,2020,5,13
35993,35994,,312,2020-05-14,37.0,4.0,2020,5,14


In [77]:
xgb.predict(df_test_final)

array([272.99905 , 281.0004  , 299.00058 , ...,  36.00076 ,  36.99918 ,
        37.001465], dtype=float32)

In [79]:
df_test_final = df_test[["Country_Region", "Year", "Month", "Day"]]

df_test["predict_confirm"] = xgb.predict(df_test_final)
df_test

,ForecastId,Province_State,Country_Region,Date,Year,Month,Day,predict_confirm
0,1,,0,2020-04-02,2020,4,2,272.999054
1,2,,0,2020-04-03,2020,4,3,281.000397
2,3,,0,2020-04-04,2020,4,4,299.000580
3,4,,0,2020-04-05,2020,4,5,349.000519
4,5,,0,2020-04-06,2020,4,6,367.000305
...,...,...,...,...,...,...,...,...
13454,13455,,312,2020-05-10,2020,5,10,36.000092
13455,13456,,312,2020-05-11,2020,5,11,35.999012
13456,13457,,312,2020-05-12,2020,5,12,36.000759
13457,13458,,312,2020-05-13,2020,5,13,36.999180
